<a href="https://colab.research.google.com/github/Elias792/Projet-Pi2-FitBuddy/blob/main/1_4_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv('data.csv')

In [11]:
df

,Timestamp,X_Acceleration,Y_Acceleration,Z_Acceleration
0,2025-03-31 00:00:00.000,0.00,0.00,1.00
1,2025-03-31 00:00:00.100,0.05,0.02,0.99
2,2025-03-31 00:00:00.200,0.10,0.04,0.98
3,2025-03-31 00:00:00.300,0.15,0.06,0.97
4,2025-03-31 00:00:00.400,0.20,0.08,0.96
...,...,...,...,...
102,2025-03-31 00:00:10.200,0.50,0.20,0.90
103,2025-03-31 00:00:10.300,0.43,0.17,0.90
104,2025-03-31 00:00:10.400,0.36,0.14,0.91
105,2025-03-31 00:00:10.500,0.29,0.11,0.92


In [12]:
columns = df.columns
columns

Index(['Timestamp', 'X_Acceleration', 'Y_Acceleration', 'Z_Acceleration'], dtype='object')

In [13]:
# Simulate UWB position data (assuming random walk with noise)
np.random.seed(42)  # For reproducibility
num_samples = len(df)

# Assume the object starts at (0,0,0) and moves randomly
uwb_x = np.cumsum(np.random.normal(0, 0.1, num_samples))  # Simulated X position
uwb_y = np.cumsum(np.random.normal(0, 0.1, num_samples))  # Simulated Y position
uwb_z = np.cumsum(np.random.normal(0, 0.1, num_samples))  # Simulated Z position

# Add simulated UWB data to the dataframe
df["UWB_X"] = uwb_x
df["UWB_Y"] = uwb_y
df["UWB_Z"] = uwb_z

# Display first few rows with UWB data
df.head()

,Timestamp,X_Acceleration,Y_Acceleration,Z_Acceleration,UWB_X,UWB_Y,UWB_Z
0,2025-03-31 00:00:00.000,0.00,0.00,1.00,0.049671,0.017458,-0.031527
1,2025-03-31 00:00:00.100,0.05,0.02,0.99,0.035845,0.043213,0.044370
2,2025-03-31 00:00:00.200,0.10,0.04,0.98,0.100614,0.035768,-0.032913
3,2025-03-31 00:00:00.300,0.15,0.06,0.97,0.252917,-0.156109,-0.056594
4,2025-03-31 00:00:00.400,0.20,0.08,0.96,0.229501,-0.158760,-0.105131


In [14]:
!pip install filterpy

In [20]:
import numpy as np
import pandas as pd
from filterpy.kalman import ExtendedKalmanFilter as EKF
from filterpy.kalman import UnscentedKalmanFilter as UKF, MerweScaledSigmaPoints
from filterpy.common import Q_discrete_white_noise


# Define state: [x, y, z]

def f(state, dt):
    """ State transition function for position update """
    x, y, z = state
    return np.array([x, y, z])

def h(state):
    """ Measurement function: extracts position (UWB) """
    return state[:3]

def jacobian_h(state):
    """ Jacobian of h """
    return np.eye(3)  # Since h(state) = state[:3], the Jacobian is an identity matrix

# Initialize EKF
dt = 0.1  # Time step
initial_state = [0, 0, 0]
ekf = EKF(dim_x=3, dim_z=3)
ekf.x = np.array(initial_state)
ekf.F = jacobian_f(ekf.x, dt)
ekf.H = np.eye(3) # You might not need this line if you are providing HJacobian and Hx in the update step
ekf.P *= 1
ekf.R *= 0.1
ekf.Q = Q_discrete_white_noise(dim=3, dt=dt, var=0.1)

# Initialize UKF
points = MerweScaledSigmaPoints(n=3, alpha=0.1, beta=2., kappa=-1.)
ukf = UKF(dim_x=3, dim_z=3, dt=dt, fx=f, hx=h, points=points)
ukf.x = np.array(initial_state)
ukf.P *= 1
ukf.R *= 0.1
ukf.Q = Q_discrete_white_noise(dim=3, dt=dt, var=0.1)

# Apply filters
ekf_estimates = []
ukf_estimates = []
for i in range(len(df)):
    z = df.iloc[i][["UWB_X", "UWB_Y", "UWB_Z"]].values  # UWB measurement
    ekf.predict()
    # Pass the Jacobian of the measurement function (HJacobian) and the measurement function (Hx) to ekf.update()
    ekf.update(z, HJacobian=jacobian_h, Hx=h)
    ukf.predict()
    ukf.update(z)
    ekf_estimates.append(ekf.x[:3])
    ukf_estimates.append(ukf.x[:3])

df["EKF_X"], df["EKF_Y"], df["EKF_Z"] = np.array(ekf_estimates).T
df["UKF_X"], df["UKF_Y"], df["UKF_Z"] = np.array(ukf_estimates).T

# Extract Motion Parameters
def extract_motion_parameters(df):
    df['Speed'] = np.linalg.norm(df[['X_Acceleration', 'Y_Acceleration', 'Z_Acceleration']].values, axis=1)
    df['Acceleration_Change'] = df['Speed'].diff()
    df['Phase'] = np.where(df['Acceleration_Change'] > 0, 'Concentric', 'Eccentric')

    threshold = df['Speed'].mean() / 2  # Define threshold for movement cycles
    df['Motion'] = df['Speed'] > threshold
    df['Repetition'] = (df['Motion'] != df['Motion'].shift(1)).cumsum()
    num_reps = df['Repetition'].nunique() // 2  # Count repetitions

    #rep_durations = df.groupby('Repetition')['Timestamp'].agg(lambda x: x.max() - x.min())
    # Convert 'Timestamp' column to datetime if it's not already
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Now, the aggregation should work correctly
    rep_durations = df.groupby('Repetition')['Timestamp'].agg(lambda x: x.max() - x.min())
    avg_time_per_rep = rep_durations.mean()

    amplitude = df['Speed'].max() - df['Speed'].min()
    pause_times = rep_durations[rep_durations < avg_time_per_rep / 2].sum()

    return num_reps, avg_time_per_rep, amplitude, pause_times

num_reps, avg_time_per_rep, amplitude, pause_times = extract_motion_parameters(df)

# Display extracted parameters
print(f"Number of repetitions: {num_reps}")
print(f"Average time per repetition: {avg_time_per_rep}")
print(f"Amplitude: {amplitude}")
print(f"Total pause time: {pause_times}")

Number of repetitions: 0
Average time per repetition: 0 days 00:00:10.600000
Amplitude: 0.11516009194541366
Total pause time: 0 days 00:00:00


In [22]:
df.tail(106)

,Timestamp,X_Acceleration,Y_Acceleration,Z_Acceleration,UWB_X,UWB_Y,UWB_Z,EKF_X,EKF_Y,EKF_Z,UKF_X,UKF_Y,UKF_Z,Speed,Acceleration_Change,Phase,Motion,Repetition
1,2025-03-31 00:00:00.100,0.05,0.02,0.99,0.035845,0.043213,0.044370,0.04079,0.030249,0.019693,0.04079,0.030249,0.019703,0.991464,-0.008536,Eccentric,True,1
2,2025-03-31 00:00:00.200,0.10,0.04,0.98,0.100614,0.035768,-0.032913,0.06001,0.030478,-0.012785,0.060011,0.030479,-0.012775,0.985901,-0.005563,Eccentric,True,1
3,2025-03-31 00:00:00.300,0.15,0.06,0.97,0.252917,-0.156109,-0.056594,0.106946,-0.017324,-0.046404,0.106946,-0.017324,-0.046399,0.983362,-0.002539,Eccentric,True,1
4,2025-03-31 00:00:00.400,0.20,0.08,0.96,0.229501,-0.158760,-0.105131,0.130825,-0.048088,-0.088236,0.130825,-0.048088,-0.088234,0.983870,0.000508,Concentric,True,1
5,2025-03-31 00:00:00.500,0.25,0.10,0.95,0.206088,-0.152737,-0.096943,0.143121,-0.066092,-0.098146,0.143121,-0.066092,-0.098145,0.987421,0.003551,Concentric,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2025-03-31 00:00:10.200,0.50,0.20,0.90,-1.256338,0.921905,0.266561,-0.653995,0.288788,0.275233,-0.653995,0.288788,0.275233,1.048809,0.017790,Concentric,True,1
103,2025-03-31 00:00:10.300,0.43,0.17,0.90,-1.336566,0.978994,0.235535,-0.660473,0.296983,0.290503,-0.660473,0.296983,0.290503,1.011830,-0.036979,Eccentric,True,1
104,2025-03-31 00:00:10.400,0.36,0.14,0.91,-1.352695,1.092550,0.267951,-0.666898,0.307788,0.322638,-0.666898,0.307788,0.322638,0.988585,-0.023245,Eccentric,True,1
105,2025-03-31 00:00:10.500,0.29,0.11,0.92,-1.312289,1.187950,0.254937,-0.672929,0.317116,0.332326,-0.672929,0.317116,0.332326,0.970876,-0.017709,Eccentric,True,1
